Cross Validation 1

size =  121655

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.0) & (df['Maximum_Installs'] >= 5000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_train_1.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_val_1.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(97324, 23)
(24331, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Science Kids Learning - Be Super Scientist!,com.sciencegames.ScienceKidsLearning,Educational,4.2,78871,False,Science is all about inspiring curiosity and d...,https://nutboltgames.com/privacy-policy,0,0,...,0,0,1,0,0,0,1846,science is all about inspiring curiosity and d...,1807,343
1,Phuti kismat se Bhagyashali,com.photi.bhgysl,Lifestyle,4.4,13681,False,This year has in its fold mixed fate to offer ...,https://labtest12.wixsite.com/privacypolicy,0,0,...,0,0,0,0,0,0,1513,this year has in its fold mixed fate to offer ...,1443,350


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera        16336
Location      16286
Microphone     8807
Contacts      10431
Storage       48218
Phone         15059
SMS             340
Call_Log        216
Calendar       1824
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera         4117
Location       4056
Microphone     2234
Contacts       2689
Storage       12040
Phone          3818
SMS              73
Call_Log         50
Calendar        417
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

121655


In [23]:
corpus[0]

['science',
 'inspiring',
 'curiosity',
 'discovering',
 'world',
 'learn',
 'science',
 'fun',
 'educational',
 'games',
 'science',
 'games',
 'sure',
 'fulfill',
 'sense',
 'curiosity',
 'introduce',
 'basic',
 'science',
 'concepts',
 'approachable',
 'manner',
 'reinforcing',
 'comprehension',
 'engaging',
 'stimulating',
 'activities',
 'using',
 'interactive',
 'science',
 'games',
 'creates',
 'unique',
 'learning',
 'process',
 'combined',
 'conventional',
 'topics',
 'food',
 'chain',
 'learn',
 'food',
 'chain',
 'food',
 'web',
 'energy',
 'cycles',
 'transfers',
 'living',
 'organisms',
 'lesson',
 'learn',
 'simple',
 'food',
 'chains',
 'energy',
 'passed',
 'producers',
 'consumers',
 'decomposers',
 'eats',
 'what',
 'energy',
 'life',
 'needs',
 'survive',
 'come',
 'from',
 'learn',
 'producers',
 'primary',
 'consumers',
 'secondary',
 'tertiary',
 'consumers',
 'apex',
 'predators',
 'decomposers',
 'roles',
 'play',
 'food',
 'chains',
 'microorganisms',
 'ever',


##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        science inspiring curiosity discovering world ...
1        year fold mixed fate offer fellow human beings...
2        realpi provides best interesting pi calculatio...
3        regularly helps keep track repeating tasks fix...
4        harri live highly intuitive useful team schedu...
                               ...                        
97319    toddler games year olds kids care kids right w...
97320    listening important skill japanese software he...
97321    awesome way turn plain keyboard beautiful uniq...
97322    description tv packages watch favourite progra...
97323    check app quote written arabic translated engl...
Name: Clean_Description, Length: 97324, dtype: object

In [29]:
train_sequences

[[587,
  3038,
  6265,
  6299,
  91,
  31,
  587,
  98,
  463,
  75,
  587,
  75,
  415,
  4908,
  2238,
  6265,
  3338,
  390,
  587,
  1443,
  22562,
  1899,
  19270,
  3773,
  2406,
  8727,
  616,
  39,
  618,
  587,
  75,
  2866,
  190,
  74,
  543,
  3039,
  5970,
  383,
  438,
  3449,
  31,
  438,
  3449,
  438,
  345,
  1009,
  4430,
  1802,
  1633,
  7869,
  1709,
  31,
  46,
  438,
  6782,
  1009,
  4851,
  8426,
  4339,
  58034,
  14502,
  5880,
  1009,
  83,
  422,
  6147,
  516,
  1373,
  31,
  8426,
  2383,
  4339,
  3014,
  21829,
  4339,
  1792,
  13057,
  58034,
  6266,
  69,
  438,
  6782,
  21830,
  499,
  6607,
  3146,
  139,
  12320,
  44238,
  44238,
  44238,
  17409,
  320,
  1834,
  2101,
  156,
  31,
  382,
  1633,
  3867,
  7869,
  18,
  12565,
  5469,
  12688,
  22563,
  34036,
  1203,
  353,
  296,
  2231,
  31,
  1633,
  519,
  1383,
  413,
  1028,
  3,
  408,
  438,
  688,
  181,
  1203,
  42,
  413,
  1028,
  405,
  9016,
  546,
  146,
  91,
  587,
  31,
 

In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[  587,  3038,  6265, ...,     0,     0,     0],
       [  276,  7230,  3383, ...,     0,     0,     0],
       [40031,   148,    15, ...,     0,     0,     0],
       ...,
       [  409,    64,   464, ...,     0,     0,     0],
       [ 1363,   359,  1751, ...,     0,     0,     0],
       [   94,     1,  1729, ...,     0,     0,     0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[16989,  4376, 17715, ...,     0,     0,     0],
       [  270,    22,   166, ...,     0,     0,     0],
       [  329,  4387,  1596, ...,     0,     0,     0],
       ...,
       [  165,   294,   485, ...,     0,     0,     0],
       [ 4398,   601,   267, ...,     0,     0,     0],
       [  217,  3335,    84, ...,     0,     0,     0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

science inspiring curiosity discovering world learn science fun educational games science games sure fulfill sense curiosity introduce basic science concepts approachable manner reinforcing comprehension engaging stimulating activities using interactive science games creates unique learning process combined conventional topics food chain learn food chain food web energy cycles transfers living organisms lesson learn simple food chains energy passed producers consumers decomposers eats what energy life needs survive come from learn producers primary consumers secondary tertiary consumers apex predators decomposers roles play food chains microorganisms ever wondered happens look microscope trillions trillions trillions microbes around earth maybe more learn types living micro organisms like bacteria viruses fungi algae protozoa human body health growth learn living things grow stay healthy use water food exercise provide human need stay healthy super scientist enter amazing world science

In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 194678


In [36]:
word_index

{'app': 1,
 'free': 2,
 'use': 3,
 'wallpaper': 4,
 'keyboard': 5,
 'theme': 6,
 'phone': 7,
 'application': 8,
 'features': 9,
 'new': 10,
 'get': 11,
 'wallpapers': 12,
 'time': 13,
 'also': 14,
 'best': 15,
 'screen': 16,
 'one': 17,
 'like': 18,
 'make': 19,
 'download': 20,
 'easy': 21,
 'android': 22,
 'launcher': 23,
 'us': 24,
 'english': 25,
 'live': 26,
 'mobile': 27,
 'share': 28,
 'find': 29,
 'help': 30,
 'learn': 31,
 'hd': 32,
 'set': 33,
 'love': 34,
 'device': 35,
 'video': 36,
 'please': 37,
 'want': 38,
 'using': 39,
 'apps': 40,
 'images': 41,
 'need': 42,
 'support': 43,
 'information': 44,
 'home': 45,
 'simple': 46,
 'themes': 47,
 'many': 48,
 'friends': 49,
 'available': 50,
 'enjoy': 51,
 'different': 52,
 'text': 53,
 'language': 54,
 'data': 55,
 'access': 56,
 'day': 57,
 'beautiful': 58,
 'game': 59,
 'easily': 60,
 'add': 61,
 'user': 62,
 'online': 63,
 'way': 64,
 'you': 65,
 'choose': 66,
 'words': 67,
 'create': 68,
 'play': 69,
 'every': 70,
 'status

In [37]:
word_index["reason"]

2390

In [38]:
print(validation_sequences[0])

[16989, 4376, 17715, 12, 148, 41, 17715, 772, 3, 109, 1, 66, 112, 275, 210, 339, 131, 210, 33, 4, 131, 107, 4, 2107, 210, 72, 131, 107, 275, 785, 210, 28, 131, 107, 275, 1439, 340, 1, 267, 16989, 4376, 17715, 772, 1337, 137, 1, 856, 1602, 1152, 2127, 1636, 1743, 530, 524, 1467, 2131, 859, 645, 41, 8, 1234, 320, 345, 11203, 524, 37, 245, 2, 99, 24, 583, 504]


In [39]:
word_index["listen"]

388

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.30414   , -0.37029999, -0.15881   , ..., -0.37830999,
        -0.42910001,  0.0030023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
word_index["reason"]

2390

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(97324, 600)
(97324, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(24331, 600)
(24331, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

97324
9
{0: 0.6619599521166613, 1: 0.6639922496486417, 2: 1.227861675687269, 3: 1.036696172732986, 4: 0.2242684843373383, 5: 0.7180940153913127, 6: 31.805228758169935, 7: 50.06378600823045, 8: 5.928606237816764}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    58403700    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    58403700    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    58403700    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
3042/3042 [==============================] - 591s 191ms/step - loss: 0.1601 - accuracy: 0.2726 - val_loss: 0.2354 - val_accuracy: 0.2908
Epoch 2/10
3042/3042 [==============================] - 579s 190ms/step - loss: 0.1320 - accuracy: 0.2953 - val_loss: 0.2162 - val_accuracy: 0.3012
Epoch 3/10
3042/3042 [==============================] - 575s 189ms/step - loss: 0.1113 - accuracy: 0.2991 - val_loss: 0.2143 - val_accuracy: 0.2920
Epoch 4/10
3042/3042 [==============================] - 611s 201ms/step - loss: 0.0920 - accuracy: 0.3043 - val_loss: 0.3058 - val_accuracy: 0.3279
Epoch 5/10
3042/3042 [==============================] - 580s 191ms/step - loss: 0.0808 - accuracy: 0.3067 - val_loss: 0.2296 - val_accuracy: 0.2836
Epoch 6/10
3042/3042 [==============================] - 579s 190ms/step - loss: 0.0710 - accuracy: 0.3142 - val_loss: 0.2501 - val_accuracy: 0.2928
Epoch 7/10
3042/3042 [==============================] - 577s 190ms/step - loss: 0.0626 - accuracy: 0.3285 - val_

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2628, 1969, 29, 15, 2628, 8936, 442, 682, 2399, 1, 1, 194, 13, 15, 442, 2628, 1001, 442, 8936, 140, 442, 2628, 124, 15, 442, 2628, 1001, 442, 8936, 18, 27345, 40386, 13417, 14266, 30387, 327, 14, 194, 599, 442, 2628, 1712, 442, 2628, 442, 10059, 140, 442, 633, 442, 633, 1, 194, 15, 499, 26

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[1.7155601e-03, 1.9788116e-01, 7.3202154e-06, 3.1178264e-05,
        9.1992807e-01, 1.3370777e-02, 5.6859935e-09, 1.6216533e-11,
        2.4194290e-07],
       [9.1632555e-06, 7.4662952e-05, 2.8009633e-06, 7.1229316e-08,
        5.1643199e-04, 4.7557132e-05, 5.3581291e-09, 7.9488187e-14,
        5.0214834e-07],
       [3.2079360e-03, 7.6483193e-05, 5.5794029e-05, 2.2231521e-04,
        1.1376147e-02, 6.0750917e-03, 2.7444496e-06, 6.2764980e-09,
        1.1180309e-05],
       [5.5317977e-04, 2.7432665e-02, 4.3785229e-04, 7.3130205e-03,
        3.0654755e-01, 1.0906724e-01, 2.8766966e-05, 1.4940041e-06,
        2.4037139e-04],
       [2.3547021e-01, 6.7770976e-04, 1.0300899e-02, 2.3943445e-02,
        2.9127041e-01, 2.8165886e-01, 7.0378861e-05, 7.3903248e-06,
        2.2966124e-04],
       [9.1697323e-01, 2.1259764e-02, 3.1208762e-03, 1.8574178e-02,
        9.9955362e-01, 1.8323801e-01, 5.9447956e-07, 2.9520173e-08,
        2.5845462e-05],
       [8.5569566e-01, 8.1453925e-01, 3.

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[1.7155601e-03 1.9788116e-01 7.3202154e-06 ... 5.6859935e-09
  1.6216533e-11 2.4194290e-07]
 [9.1632555e-06 7.4662952e-05 2.8009633e-06 ... 5.3581291e-09
  7.9488187e-14 5.0214834e-07]
 [3.2079360e-03 7.6483193e-05 5.5794029e-05 ... 2.7444496e-06
  6.2764980e-09 1.1180309e-05]
 ...
 [1.5112378e-01 3.3330366e-01 6.6462453e-03 ... 2.5858416e-04
  5.3364667e-05 1.1491369e-03]
 [1.4109081e-03 9.9573201e-01 9.9979347e-01 ... 1.9112387e-10
  5.3482475e-12 8.2087681e-05]
 [6.6516513e-04 3.8369559e-04 5.3279648e-05 ... 2.3342231e-07
  1.0101705e-10 1.9653256e-05]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.5407 with F-Score: 0.6812
permission  1
Length of sequence: 10000
Best Threshold: 0.6885 with F-Score: 0.5824
permission  2
Length of sequence: 10000
Best Threshold: 0.1591 with F-Score: 0.5707
permission  3
Length of sequence: 10000
Best Threshold: 0.1689 with F-Score: 0.5327
permission  4
Length of sequence: 10000
Best Threshold: 0.1718 with F-Score: 0.7966
permission  5
Length of sequence: 10000
Best Threshold: 0.1291 with F-Score: 0.5187
permission  6
Length of sequence: 10000
Best Threshold: 0.0357 with F-Score: 0.1356
permission  7
Length of sequence: 10000
Best Threshold: 0.3956 with F-Score: 0.2857
permission  8
Length of sequence: 10000
Best Threshold: 0.1689 with F-Score: 0.356
-------------------------------------
optimal threshold tuning for f-score
[0.5407, 0.6885, 0.1591, 0.1689, 0.1718, 0.1291, 0.0357, 0.3956, 0.1689]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.11240000277757645 with G-Mean: 0.837
FPR: 0.1596, TPR: 0.8336
permission  1
Best Threshold: 0.04670000076293945 with G-Mean: 0.7715
FPR: 0.239, TPR: 0.7821
permission  2
Best Threshold: 0.006500000134110451 with G-Mean: 0.8206
FPR: 0.139, TPR: 0.7821
permission  3
Best Threshold: 0.011800000444054604 with G-Mean: 0.7978
FPR: 0.1712, TPR: 0.768
permission  4
Best Threshold: 0.6550999879837036 with G-Mean: 0.7797
FPR: 0.1782, TPR: 0.7398
permission  5
Best Threshold: 0.022299999371170998 with G-Mean: 0.7726
FPR: 0.2684, TPR: 0.816
permission  6
Best Threshold: 0.0 with G-Mean: 0.7966
FPR: 0.2244, TPR: 0.8182
permission  7
Best Threshold: 0.0003000000142492354 with G-Mean: 0.8816
FPR: 0.0673, TPR: 0.8333
permission  8
Best Threshold: 0.002899999963119626 with G-Mean: 0.8204
FPR: 0.1703, TPR: 0.8111
-------------------------------------
ROC curve with G-mean threshold tuning
[0.1124, 0.0467, 0.0065, 0.0118, 0.6551, 0.0223, 0.0, 0.0003, 0.0029]


## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV1_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6812
  Location  : 0.5824
  Microphone: 0.5707
  Contacts  : 0.5327
  Storage   : 0.7966
  Phone     : 0.5187
  SMS       : 0.1356
  Call_Log  : 0.2857
  Calendar  : 0.3560

  Average F1 score: 0.4955


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV1_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8927
  Location  : 0.8778
  Microphone: 0.9245
  Contacts  : 0.8919
  Storage   : 0.7736
  Phone     : 0.8410
  SMS       : 0.9890
  Call_Log  : 0.9989
  Calendar  : 0.9734

  Average F1 (micro) score: 0.9070


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV1_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8363
  Location  : 0.7716
  Microphone: 0.8205
  Contacts  : 0.7974
  Storage   : 0.7806
  Phone     : 0.7735
  SMS       : 0.5000
  Call_Log  : 0.8826
  Calendar  : 0.8200

  Average ROC_AUC score: 0.7758


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.7648 with F-Score: 0.4204
permission  1
Length of sequence: 10000
Best Threshold: 0.4252 with F-Score: 0.5405
permission  2
Length of sequence: 10000
Best Threshold: 0.2274 with F-Score: 0.527
permission  3
Length of sequence: 10000
Best Threshold: 0.0456 with F-Score: 0.5947
permission  4
Length of sequence: 10000
Best Threshold: 0.0384 with F-Score: 0.5884
permission  5
Length of sequence: 10000
Best Threshold: 0.7073 with F-Score: 0.4305
permission  6
Length of sequence: 10000
Best Threshold: 0.0027 with F-Score: 0.5489
permission  7
Length of sequence: 10000
Best Threshold: 0.0352 with F-Score: 0.3568
permission  8
Length of sequence: 10000
Best Threshold: 0.2219 with F-Score: 0.4916
-------------------------------------
optimal threshold tuning for f-score
[0.7648, 0.4252, 0.2274, 0.0456, 0.0384, 0.7073, 0.0027, 0.0352, 0.2219]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.1590999960899353 with G-Mean: 0.6633
FPR: 0.3662, TPR: 0.6943
permission  1
Best Threshold: 0.36489999294281006 with G-Mean: 0.7183
FPR: 0.2234, TPR: 0.6645
permission  2
Best Threshold: 0.024000000208616257 with G-Mean: 0.7739
FPR: 0.1793, TPR: 0.7297
permission  3
Best Threshold: 0.04580000042915344 with G-Mean: 0.7046
FPR: 0.2605, TPR: 0.6713
permission  4
Best Threshold: 0.7756999731063843 with G-Mean: 0.5845
FPR: 0.3859, TPR: 0.5563
permission  5
Best Threshold: 0.10080000013113022 with G-Mean: 0.747
FPR: 0.3638, TPR: 0.8772
permission  6
Best Threshold: 0.00019999999494757503 with G-Mean: 0.78
FPR: 0.2413, TPR: 0.8019
permission  7
Best Threshold: 0.0 with G-Mean: 0.753
FPR: 0.2838, TPR: 0.7917
permission  8
Best Threshold: 0.015699999406933784 with G-Mean: 0.8207
FPR: 0.213, TPR: 0.8558
-------------------------------------
ROC curve with G-mean threshold tuning
[0.1591, 0.3649, 0.024, 0.0458, 0.7757, 0.1008, 0.0002, 0.0, 0.0157]


#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV1_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4204
  Location  : 0.5405
  Microphone: 0.5270
  Contacts  : 0.5947
  Storage   : 0.5884
  Phone     : 0.4305
  SMS       : 0.5489
  Call_Log  : 0.3568
  Calendar  : 0.4916

  Average F1 score: 0.4998


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV1_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8147
  Location  : 0.7631
  Microphone: 0.9010
  Contacts  : 0.7185
  Storage   : 0.4894
  Phone     : 0.9102
  SMS       : 0.8465
  Call_Log  : 0.9158
  Calendar  : 0.9356

  Average F1 (micro) score: 0.8105


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV1_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.6619
  Location  : 0.7205
  Microphone: 0.7752
  Contacts  : 0.7042
  Storage   : 0.5852
  Phone     : 0.7523
  SMS       : 0.7788
  Call_Log  : 0.5000
  Calendar  : 0.8166

  Average ROC_AUC score: 0.6994
